In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.common.exceptions import NoSuchElementException, TimeoutException, InvalidArgumentException
# from selenium.webdriver.support.ui import 

from selenium.webdriver.support.ui import Select
import time

In [2]:
import pandas as pd

# Connect app

In [5]:
browser = webdriver.Chrome(executable_path=r'.\chromedriver.exe')

# Import excel

In [5]:
df = pd.ExcelFile('PEMETAAN KELAS 20212022 NEW - Copy.xlsx')


In [9]:
from datetime import datetime
date_time_str = '18/01/19 01:55:19'
date_time_obj = datetime.strptime(date_time_str, '%d/%m/%y %H:%M:%S')

In [71]:
df_try = df.parse(df.sheet_names[0])
df_try = df_try[df_try.No.notna()]

In [8]:
# format ini penting!!!!!!!!!!!!!!!!!!!
format_month = {
    'Januari'    :'01',
    'Februari'   :'02', 
    'Maret'      :'03', 
    'April'      :'04', 
    'Mei'        :'05', 
    'Juni'       :'06', 
    'Juli'       :'07', 
    'Agustus'    :'08', 
    'September'  :'09', 
    'Oktober'    :'10', 
    'November'   :'11', 
    'Desember'   :'12',
    ' '          :'/'
}

fix_agama = {
    'Khatolik':'Katolik',
    'Kong Hu Cu': 'Konghucu'
}

In [199]:
df_try['Pada Tanggal'] = df_try['Pada Tanggal'].replace(format_month, regex=True)
df_try['Tempat Lahir'] = df_try['Tempat Tanggal Lahir'].str.split(',').str[0]
df_try['Tanggal Lahir'] = df_try['Tempat Tanggal Lahir'].str.split(',').str[1].str[1:]
df_try['Tanggal Lahir'] = df_try['Tanggal Lahir'].replace(format_month, regex=True)
df_try['Anak Ke'] = df_try['Anak Ke -'].str[0]
df_try['Agama'] = df_try['Agama'].replace(fix_agama, regex=True)

# Satu file SMP

In [18]:
df = pd.read_excel('data siswa yang belum terinput (SD).xlsx',dtype='str')

In [21]:
browser.get('https://sd.mitrakasih.online/student/create')
if 'login' in browser.current_url:
    browser.find_element(By.XPATH,'//*[@id="email"]').clear()
    browser.find_element(By.XPATH,'//*[@id="password"]').clear()
    browser.find_element(By.XPATH,'//*[@id="email"]').send_keys("")
    browser.find_element(By.XPATH,'//*[@id="password"]').send_keys("")
    browser.find_element(By.XPATH,'/html/body/div/div/main/div/div/div/div/div/div/form/div[6]/div/button').click()
    pass


In [22]:
df_try = df.copy()
df_try = df_try[df_try.iloc[:,0].notna()]
# df_try = df_try.replace({'-':''})
df_try = df_try.fillna('')


df_try['Pada Tanggal'] = df_try['Pada Tanggal'].replace(format_month, regex=True)
df_try['Tempat Lahir'] = df_try['Tempat Tanggal Lahir'].str.split(',').str[0]
df_try['Tanggal Lahir'] = df_try['Tempat Tanggal Lahir'].str.split(',').str[1].str[1:]
df_try['Tanggal Lahir'] = df_try['Tanggal Lahir'].replace(format_month, regex=True)
df_try['Jenis Kelamin'] = df_try['Jenis Kelamin'].str.replace('Laki-laki','Laki-Laki')
df_try['Anak Ke'] = df_try['Anak Ke -'].str[0]
df_try['Agama'] = df_try['Agama'].replace(fix_agama, regex=True)
df_try['Jenis Kelamin'] = df_try['Jenis Kelamin'].str.replace(' ','')

for idx_df,baris in df_try.iterrows():
    browser.get('https://sd.mitrakasih.online/student/create')
    wait = WebDriverWait(browser, 30)

    # pilih jenis pendaftaran
    select = Select(browser.find_element(By.XPATH,'//*[@id="card0"]/div/div/div[1]/select'))
    select.select_by_value('1')
    # Pendaftaran Siswa
    browser.find_element(By.XPATH,'//*[@id="accordion"]/div/button').click()
    time.sleep(0.2)
    date= wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@name="date_school"]')))
    browser.execute_script(f"arguments[0].removeAttribute('readonly')", date)
    browser.find_element(By.XPATH,'//*[@name="date_school"]').clear()
    browser.find_element(By.XPATH,'//*[@name="date_school"]').send_keys(baris['Pada Tanggal'])
    browser.find_element(By.XPATH,'//*[@name="school_before"]').clear()
    browser.find_element(By.XPATH,'//*[@name="school_before"]').send_keys(baris['Nama Sekolah Asal'])
    browser.find_element(By.XPATH,'//*[@name="school_address"]').clear()
    browser.find_element(By.XPATH,'//*[@name="school_address"]').send_keys(baris['Alamat Sekolah Asal'])


    # Data Pribadi Siswa
    browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[2]/button').click()
    time.sleep(0.2)
    browser.find_element(By.XPATH,'//*[@name="name"]').clear()
    browser.find_element(By.XPATH,'//*[@name="name"]').send_keys(baris['Nama'])

    try:
        NIS = int(baris['No Induk'])
    except:
        NIS = str(baris['No Induk'])
    browser.find_element(By.XPATH,'//*[@name="nis"]').clear()
    browser.find_element(By.XPATH,'//*[@name="nis"]').send_keys(NIS)

    try:
        NISN = int(baris['NISN'])
    except:
        NISN = str(baris['NISN'])
    browser.find_element(By.XPATH,'//*[@name="nisn"]').clear()
    browser.find_element(By.XPATH,'//*[@name="nisn"]').send_keys(NISN)
    browser.find_element(By.XPATH,'//*[@name="place_born"]').clear()
    browser.find_element(By.XPATH,'//*[@name="place_born"]').send_keys(baris['Tempat Lahir'])
    date= wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@name="birth_born"]')))
    browser.execute_script(f"arguments[0].removeAttribute('readonly')", date)
    browser.find_element(By.XPATH,'//*[@name="birth_born"]').clear()
    browser.find_element(By.XPATH,'//*[@name="birth_born"]').send_keys(baris['Tanggal Lahir'])
    select = Select(browser.find_element(By.XPATH,'//*[@name="gender"]'))
    select.select_by_visible_text(baris['Jenis Kelamin'].replace('laki','Laki'))
    browser.find_element(By.XPATH,'//*[@name="address"]').clear()
    browser.find_element(By.XPATH,'//*[@name="address"]').send_keys(baris['Nama Ibu'])
    select = Select(browser.find_element(By.XPATH,'//*[@name="child_order"]'))
    select.select_by_visible_text(baris['Anak Ke'])

    # Data Ibu Kandung
    browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[3]/button').click()
    time.sleep(0.2)
    browser.find_element(By.XPATH,'//*[@name="parent[1][name]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[1][name]"]').send_keys(baris['Nama Ibu'])
    browser.find_element(By.XPATH,'//*[@name="parent[1][phone_number]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[1][phone_number]"]').send_keys(str(baris['Telp Ibu']))
    browser.find_element(By.XPATH,'//*[@name="parent[1][work]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[1][work]"]').send_keys(baris['Pekerjaan Ibu'])

    # Data Ayah Kandung
    browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[4]/button').click()
    time.sleep(0.2)
    browser.find_element(By.XPATH,'//*[@name="parent[2][name]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[2][name]"]').send_keys(baris['Nama Ayah'])
    browser.find_element(By.XPATH,'//*[@name="parent[2][phone_number]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[2][phone_number]"]').send_keys(str(baris['Telp Ayah']))
    browser.find_element(By.XPATH,'//*[@name="parent[2][work]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[2][work]"]').send_keys(baris['Pekerjaan Ayah'])

    # Data Wali
    browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[5]/button').click()
    time.sleep(0.2)
    browser.find_element(By.XPATH,'//*[@name="parent[3][name]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[3][name]"]').send_keys(baris['Nama Wali'])
    browser.find_element(By.XPATH,'//*[@name="parent[3][phone_number]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[3][phone_number]"]').send_keys(str(baris['Telp Wali']))
    browser.find_element(By.XPATH,'//*[@name="parent[3][work]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[3][work]"]').send_keys(baris['Pekerjaan Wali'])

    browser.find_element(By.XPATH,'//*[@id="content"]/form/div/div[2]/button').click()

# Iterasi SD

In [307]:
df = pd.ExcelFile('9528D500.xlsx')

In [290]:
range_sd = range(0,6)
list_sd = [s for s in df.sheet_names for i in range_sd if 'P'+str(i+1) in s]
print(list_sd)

['P1A', 'P1B', 'P1C', 'P2A', 'P2B', 'P2C', 'P3 A', 'P3 B', 'P3 C', 'P4A', 'P4B', 'P4C', 'P5 A', 'P5B 3', 'P5 C ', 'P5 D', 'P6A', 'P6B']


In [298]:
kelas_sd

'P3 B'

In [303]:
browser.get('https://sd.mitrakasih.online/student/create')
if 'login' in browser.current_url:
    browser.find_element(By.XPATH,'//*[@id="email"]').clear()
    browser.find_element(By.XPATH,'//*[@id="password"]').clear()
    browser.find_element(By.XPATH,'//*[@id="email"]').send_keys("")
    browser.find_element(By.XPATH,'//*[@id="password"]').send_keys("")
    browser.find_element(By.XPATH,'/html/body/div/div/main/div/div/div/div/div/div/form/div[6]/div/button').click()
    pass

#list sd sementara. kalo dah kelar dan dk ad eror, hapus aja
list_sd = ['P4C', 'P5 A', 'P5B 3', 'P5 C ', 'P5 D', 'P6A', 'P6B']

# iterasi semua sheet
for kelas_sd in list_sd:
    print(kelas_sd)
    df_try = df.parse(kelas_sd)
    df_try = df_try[df_try.iloc[:,0].notna()]
    df_try = df_try.replace({'-':''})
    df_try = df_try.fillna('')
    
    
    df_try['Pada Tanggal'] = df_try['Pada Tanggal'].replace(format_month, regex=True)
    df_try['Tempat Lahir'] = df_try['Tempat Tanggal Lahir'].str.split(',').str[0]
    df_try['Tanggal Lahir'] = df_try['Tempat Tanggal Lahir'].str.split(',').str[1].str[1:]
    df_try['Tanggal Lahir'] = df_try['Tanggal Lahir'].replace(format_month, regex=True)
    df_try['Jenis Kelamin'] = df_try['Jenis Kelamin'].str.replace('Laki-laki','Laki-Laki')
    df_try['Anak Ke'] = df_try['Anak Ke -'].str[0]
    df_try['Agama'] = df_try['Agama'].replace(fix_agama, regex=True)
    df_try['Jenis Kelamin'] = df_try['Jenis Kelamin'].str.replace(' ','')
    
    for idx_df,baris in df_try.iterrows():
        browser.get('https://sd.mitrakasih.online/student/create')
        wait = WebDriverWait(browser, 30)

        # pilih jenis pendaftaran
        select = Select(browser.find_element(By.XPATH,'//*[@id="card0"]/div/div/div[1]/select'))
        select.select_by_value('1')
        # Pendaftaran Siswa
        browser.find_element(By.XPATH,'//*[@id="accordion"]/div/button').click()
        time.sleep(0.2)
        date= wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@name="date_school"]')))
        browser.execute_script(f"arguments[0].removeAttribute('readonly')", date)
        browser.find_element(By.XPATH,'//*[@name="date_school"]').clear()
        browser.find_element(By.XPATH,'//*[@name="date_school"]').send_keys(baris['Pada Tanggal'])
        browser.find_element(By.XPATH,'//*[@name="school_before"]').clear()
        browser.find_element(By.XPATH,'//*[@name="school_before"]').send_keys(baris['Nama Sekolah Asal'])
        browser.find_element(By.XPATH,'//*[@name="school_address"]').clear()
        browser.find_element(By.XPATH,'//*[@name="school_address"]').send_keys(baris['Alamat Sekolah Asal'])


        # Data Pribadi Siswa
        browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[2]/button').click()
        time.sleep(0.2)
        browser.find_element(By.XPATH,'//*[@name="name"]').clear()
        browser.find_element(By.XPATH,'//*[@name="name"]').send_keys(baris['Nama'])
        
        try:
            NIS = int(baris['No Induk'])
        except:
            NIS = str(baris['No Induk'])
        browser.find_element(By.XPATH,'//*[@name="nis"]').clear()
        browser.find_element(By.XPATH,'//*[@name="nis"]').send_keys(NIS)
        
        try:
            NISN = int(baris['NISN'])
        except:
            NISN = str(baris['NISN'])
        browser.find_element(By.XPATH,'//*[@name="nisn"]').clear()
        browser.find_element(By.XPATH,'//*[@name="nisn"]').send_keys(NISN)
        browser.find_element(By.XPATH,'//*[@name="place_born"]').clear()
        browser.find_element(By.XPATH,'//*[@name="place_born"]').send_keys(baris['Tempat Lahir'])
        date= wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@name="birth_born"]')))
        browser.execute_script(f"arguments[0].removeAttribute('readonly')", date)
        browser.find_element(By.XPATH,'//*[@name="birth_born"]').clear()
        browser.find_element(By.XPATH,'//*[@name="birth_born"]').send_keys(baris['Tanggal Lahir'])
        select = Select(browser.find_element(By.XPATH,'//*[@name="gender"]'))
        select.select_by_visible_text(baris['Jenis Kelamin'].replace('laki','Laki'))
        browser.find_element(By.XPATH,'//*[@name="address"]').clear()
        browser.find_element(By.XPATH,'//*[@name="address"]').send_keys(baris['Nama Ibu'])
        select = Select(browser.find_element(By.XPATH,'//*[@name="child_order"]'))
        select.select_by_visible_text(baris['Anak Ke'])

        # Data Ibu Kandung
        browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[3]/button').click()
        time.sleep(0.2)
        browser.find_element(By.XPATH,'//*[@name="parent[1][name]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[1][name]"]').send_keys(baris['Nama Ibu'])
        browser.find_element(By.XPATH,'//*[@name="parent[1][phone_number]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[1][phone_number]"]').send_keys(str(baris['Telp Ibu']))
        browser.find_element(By.XPATH,'//*[@name="parent[1][work]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[1][work]"]').send_keys(baris['Pekerjaan Ibu'])

        # Data Ayah Kandung
        browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[4]/button').click()
        time.sleep(0.2)
        browser.find_element(By.XPATH,'//*[@name="parent[2][name]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[2][name]"]').send_keys(baris['Nama Ayah'])
        browser.find_element(By.XPATH,'//*[@name="parent[2][phone_number]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[2][phone_number]"]').send_keys(str(baris['Telp Ayah']))
        browser.find_element(By.XPATH,'//*[@name="parent[2][work]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[2][work]"]').send_keys(baris['Pekerjaan Ayah'])

        # Data Wali
        browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[5]/button').click()
        time.sleep(0.2)
        browser.find_element(By.XPATH,'//*[@name="parent[3][name]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[3][name]"]').send_keys(baris['Nama Wali'])
        browser.find_element(By.XPATH,'//*[@name="parent[3][phone_number]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[3][phone_number]"]').send_keys(str(baris['Telp Wali']))
        browser.find_element(By.XPATH,'//*[@name="parent[3][work]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[3][work]"]').send_keys(baris['Pekerjaan Wali'])

        browser.find_element(By.XPATH,'//*[@id="content"]/form/div/div[2]/button').click()

P4C
P5 A
P5B 3
P5 C 
P5 D
P6A
P6B


In [302]:
for idx_df,baris in df_try[15:].iterrows():
    browser.get('https://sd.mitrakasih.online/student/create')
    if 'login' in browser.current_url:
        browser.find_element(By.XPATH,'//*[@id="email"]').clear()
        browser.find_element(By.XPATH,'//*[@id="password"]').clear()
        browser.find_element(By.XPATH,'//*[@id="email"]').send_keys("")
        browser.find_element(By.XPATH,'//*[@id="password"]').send_keys("")
        browser.find_element(By.XPATH,'/html/body/div/div/main/div/div/div/div/div/div/form/div[6]/div/button').click()
        pass
    wait = WebDriverWait(browser, 30)

    # pilih jenis pendaftaran
    select = Select(browser.find_element(By.XPATH,'//*[@id="card0"]/div/div/div[1]/select'))
    select.select_by_value('1')
    # Pendaftaran Siswa
    browser.find_element(By.XPATH,'//*[@id="accordion"]/div/button').click()
    time.sleep(0.2)
    date= wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@name="date_school"]')))
    browser.execute_script(f"arguments[0].removeAttribute('readonly')", date)
    browser.find_element(By.XPATH,'//*[@name="date_school"]').clear()
    browser.find_element(By.XPATH,'//*[@name="date_school"]').send_keys(baris['Pada Tanggal'])
    browser.find_element(By.XPATH,'//*[@name="school_before"]').clear()
    browser.find_element(By.XPATH,'//*[@name="school_before"]').send_keys(baris['Nama Sekolah Asal'])
    browser.find_element(By.XPATH,'//*[@name="school_address"]').clear()
    browser.find_element(By.XPATH,'//*[@name="school_address"]').send_keys(baris['Alamat Sekolah Asal'])


    # Data Pribadi Siswa
    browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[2]/button').click()
    time.sleep(0.2)
    browser.find_element(By.XPATH,'//*[@name="name"]').clear()
    browser.find_element(By.XPATH,'//*[@name="name"]').send_keys(baris['Nama'])

    try:
        NIS = int(baris['No Induk'])
    except:
        NIS = str(baris['No Induk'])
    browser.find_element(By.XPATH,'//*[@name="nis"]').clear()
    browser.find_element(By.XPATH,'//*[@name="nis"]').send_keys(NIS)

    try:
        NISN = int(baris['NISN'])
    except:
        NISN = str(baris['NISN'])
    browser.find_element(By.XPATH,'//*[@name="nisn"]').clear()
    browser.find_element(By.XPATH,'//*[@name="nisn"]').send_keys(NISN)
    browser.find_element(By.XPATH,'//*[@name="place_born"]').clear()
    browser.find_element(By.XPATH,'//*[@name="place_born"]').send_keys(baris['Tempat Lahir'])
    date= wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@name="birth_born"]')))
    browser.execute_script(f"arguments[0].removeAttribute('readonly')", date)
    browser.find_element(By.XPATH,'//*[@name="birth_born"]').clear()
    browser.find_element(By.XPATH,'//*[@name="birth_born"]').send_keys(baris['Tanggal Lahir'])
    select = Select(browser.find_element(By.XPATH,'//*[@name="gender"]'))
    select.select_by_visible_text(baris['Jenis Kelamin'].replace('laki','Laki'))
    browser.find_element(By.XPATH,'//*[@name="address"]').clear()
    browser.find_element(By.XPATH,'//*[@name="address"]').send_keys(baris['Nama Ibu'])
    select = Select(browser.find_element(By.XPATH,'//*[@name="child_order"]'))
    select.select_by_visible_text(baris['Anak Ke'])

    # Data Ibu Kandung
    browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[3]/button').click()
    time.sleep(0.2)
    browser.find_element(By.XPATH,'//*[@name="parent[1][name]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[1][name]"]').send_keys(baris['Nama Ibu'])
    browser.find_element(By.XPATH,'//*[@name="parent[1][phone_number]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[1][phone_number]"]').send_keys(str(baris['Telp Ibu']))
    browser.find_element(By.XPATH,'//*[@name="parent[1][work]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[1][work]"]').send_keys(baris['Pekerjaan Ibu'])

    # Data Ayah Kandung
    browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[4]/button').click()
    time.sleep(0.2)
    browser.find_element(By.XPATH,'//*[@name="parent[2][name]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[2][name]"]').send_keys(baris['Nama Ayah'])
    browser.find_element(By.XPATH,'//*[@name="parent[2][phone_number]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[2][phone_number]"]').send_keys(str(baris['Telp Ayah']))
    browser.find_element(By.XPATH,'//*[@name="parent[2][work]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[2][work]"]').send_keys(baris['Pekerjaan Ayah'])

    # Data Wali
    browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[5]/button').click()
    time.sleep(0.2)
    browser.find_element(By.XPATH,'//*[@name="parent[3][name]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[3][name]"]').send_keys(baris['Nama Wali'])
    browser.find_element(By.XPATH,'//*[@name="parent[3][phone_number]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[3][phone_number]"]').send_keys(str(baris['Telp Wali']))
    browser.find_element(By.XPATH,'//*[@name="parent[3][work]"]').clear()
    browser.find_element(By.XPATH,'//*[@name="parent[3][work]"]').send_keys(baris['Pekerjaan Wali'])

    browser.find_element(By.XPATH,'//*[@id="content"]/form/div/div[2]/button').click()

In [271]:
baris['Jenis Kelamin'].replace('laki','Laki')

'Laki-Laki'

In [292]:
idx_df,kelas_sd

(19, 'P3 B')

# Iterasi SMP

In [317]:
df = pd.ExcelFile('57B6E500.xlsx')

In [308]:
range_smp = range(6,9)
list_smp = [s for s in df.sheet_names for i in range_smp if 'G'+str(i+1) in s]
print(list_smp)

['G7 A ', 'G7 B', 'G8 A ', 'G8 B', 'G9 A', 'G9 B']


In [ ]:
browser.get('https://smp.mitrakasih.online/student/create')


#list sd sementara. kalo dah kelar dan dk ad eror, hapus aja
list_smp = ['G7 B', 'G8 A ', 'G8 B', 'G9 A', 'G9 B']

# iterasi semua sheet
for kelas_sd in list_smp:
    print(kelas_sd)
    if 'login' in browser.current_url:
        browser.find_element(By.XPATH,'//*[@id="email"]').clear()
        browser.find_element(By.XPATH,'//*[@id="password"]').clear()
        browser.find_element(By.XPATH,'//*[@id="email"]').send_keys("")
        browser.find_element(By.XPATH,'//*[@id="password"]').send_keys("")
        browser.find_element(By.XPATH,'/html/body/div/div/main/div/div/div/div/div/div/form/div[6]/div/button').click()
        pass
    df_try = df.parse(kelas_sd)
    df_try = df_try[df_try.iloc[:,0].notna()]
    df_try = df_try.replace({'-':''})
    df_try = df_try.fillna('')
    
    
    df_try['Pada Tanggal'] = df_try['Pada Tanggal'].replace(format_month, regex=True)
    df_try['Tempat Lahir'] = df_try['Tempat Tanggal Lahir'].str.split(',').str[0]
    df_try['Tanggal Lahir'] = df_try['Tempat Tanggal Lahir'].str.split(',').str[1].str[1:]
    df_try['Tanggal Lahir'] = df_try['Tanggal Lahir'].replace(format_month, regex=True)
    df_try['Jenis Kelamin'] = df_try['Jenis Kelamin'].str.replace('Laki-laki','Laki-Laki')
    df_try = df_try.replace({'Laki-laki':'Laki-Laki','perempuan':'Perempuan'})
    df_try['Anak Ke'] = df_try['Anak Ke -'].str[0]
    df_try['Agama'] = df_try['Agama'].replace(fix_agama, regex=True)
    df_try['Jenis Kelamin'] = df_try['Jenis Kelamin'].str.replace(' ','')
    
    for idx_df,baris in df_try.iterrows():
        print(kelas_sd,idx_df)
#         browser.get('https://smp.mitrakasih.online/student/create')
        wait = WebDriverWait(browser, 30)

        # pilih jenis pendaftaran
        select = Select(browser.find_element(By.XPATH,'//*[@id="card0"]/div/div/div[1]/select'))
        select.select_by_value('1')
        # Pendaftaran Siswa
        browser.find_element(By.XPATH,'//*[@id="accordion"]/div/button').click()
        time.sleep(0.2)
        date= wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@name="date_school"]')))
        browser.execute_script(f"arguments[0].removeAttribute('readonly')", date)
        browser.find_element(By.XPATH,'//*[@name="date_school"]').clear()
        browser.find_element(By.XPATH,'//*[@name="date_school"]').send_keys(baris['Pada Tanggal'])
        browser.find_element(By.XPATH,'//*[@name="school_before"]').clear()
        browser.find_element(By.XPATH,'//*[@name="school_before"]').send_keys(baris['Nama Sekolah Asal'])
        browser.find_element(By.XPATH,'//*[@name="school_address"]').clear()
        browser.find_element(By.XPATH,'//*[@name="school_address"]').send_keys(baris['Alamat Sekolah Asal'])


        # Data Pribadi Siswa
        browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[2]/button').click()
        time.sleep(0.2)
        browser.find_element(By.XPATH,'//*[@name="name"]').clear()
        browser.find_element(By.XPATH,'//*[@name="name"]').send_keys(baris['Nama'])
        
        try:
            NIS = int(baris['No Induk'])
        except:
            NIS = str(baris['No Induk'])
        browser.find_element(By.XPATH,'//*[@name="nis"]').clear()
        browser.find_element(By.XPATH,'//*[@name="nis"]').send_keys(NIS)
        
        try:
            NISN = int(baris['NISN'])
        except:
            NISN = str(baris['NISN'])
        browser.find_element(By.XPATH,'//*[@name="nisn"]').clear()
        browser.find_element(By.XPATH,'//*[@name="nisn"]').send_keys(NISN)
        browser.find_element(By.XPATH,'//*[@name="place_born"]').clear()
        browser.find_element(By.XPATH,'//*[@name="place_born"]').send_keys(baris['Tempat Lahir'])
        date= wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@name="birth_born"]')))
        browser.execute_script(f"arguments[0].removeAttribute('readonly')", date)
        browser.find_element(By.XPATH,'//*[@name="birth_born"]').clear()
        browser.find_element(By.XPATH,'//*[@name="birth_born"]').send_keys(baris['Tanggal Lahir'])
        select = Select(browser.find_element(By.XPATH,'//*[@name="gender"]'))
        select.select_by_visible_text(baris['Jenis Kelamin'].replace('laki','Laki'))
        browser.find_element(By.XPATH,'//*[@name="address"]').clear()
        browser.find_element(By.XPATH,'//*[@name="address"]').send_keys(baris['Nama Ibu'])
        select = Select(browser.find_element(By.XPATH,'//*[@name="child_order"]'))
        select.select_by_visible_text(baris['Anak Ke'])

        # Data Ibu Kandung
        browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[3]/button').click()
        time.sleep(0.2)
        browser.find_element(By.XPATH,'//*[@name="parent[1][name]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[1][name]"]').send_keys(baris['Nama Ibu'])
        browser.find_element(By.XPATH,'//*[@name="parent[1][phone_number]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[1][phone_number]"]').send_keys(str(baris['Telp Ibu']))
        browser.find_element(By.XPATH,'//*[@name="parent[1][work]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[1][work]"]').send_keys(baris['Pekerjaan Ibu'])

        # Data Ayah Kandung
        browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[4]/button').click()
        time.sleep(0.2)
        browser.find_element(By.XPATH,'//*[@name="parent[2][name]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[2][name]"]').send_keys(baris['Nama Ayah'])
        browser.find_element(By.XPATH,'//*[@name="parent[2][phone_number]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[2][phone_number]"]').send_keys(str(baris['Telp Ayah']))
        browser.find_element(By.XPATH,'//*[@name="parent[2][work]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[2][work]"]').send_keys(baris['Pekerjaan Ayah'])

        # Data Wali
        browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[5]/button').click()
        time.sleep(0.2)
        browser.find_element(By.XPATH,'//*[@name="parent[3][name]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[3][name]"]').send_keys(baris['Nama Wali'])
        browser.find_element(By.XPATH,'//*[@name="parent[3][phone_number]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[3][phone_number]"]').send_keys(str(baris['Telp Wali']))
        browser.find_element(By.XPATH,'//*[@name="parent[3][work]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[3][work]"]').send_keys(baris['Pekerjaan Wali'])

#         browser.find_element(By.XPATH,'//*[@id="content"]/form/div/div[2]/button').click()

G7 B
G7 B 1


# Iterasi SMA

In [10]:
df = pd.ExcelFile('B693F500.xlsx')

In [14]:
range_sma = range(9,12)
list_sma = [s for s in df.sheet_names for i in range_sma if 'G'+str(i+1) in s]
print(list_sma)

['G10', 'G11']


In [15]:
idx_df,kelas_sd

(13, 'G11')

In [20]:
link_url ='https://sma.mitrakasih.online/student/create'
browser.get(link_url)

In [21]:
#list sd sementara. kalo dah kelar dan dk ad eror, hapus aja
# list_sma = ['G11']

# iterasi semua sheet
for kelas_sd in list_sma:
    if 'login' in browser.current_url:
        browser.find_element(By.XPATH,'//*[@id="email"]').clear()
        browser.find_element(By.XPATH,'//*[@id="password"]').clear()
        browser.find_element(By.XPATH,'//*[@id="email"]').send_keys("")
        browser.find_element(By.XPATH,'//*[@id="password"]').send_keys("")
        browser.find_element(By.XPATH,'/html/body/div/div/main/div/div/div/div/div/div/form/div[6]/div/button').click()
        pass
    print(kelas_sd)
    df_try = df.parse(kelas_sd)
    df_try = df_try[df_try.iloc[:,0].notna()]
    df_try = df_try.replace({'-':''})
    df_try = df_try.fillna('')
    
    
    df_try['Pada Tanggal'] = df_try['Pada Tanggal'].replace(format_month, regex=True)
    df_try['Tempat Lahir'] = df_try['Tempat Tanggal Lahir'].str.split(',').str[0]
    df_try['Tanggal Lahir'] = df_try['Tempat Tanggal Lahir'].str.split(',').str[1].str[1:]
    df_try['Tanggal Lahir'] = df_try['Tanggal Lahir'].replace(format_month, regex=True)
    df_try['Jenis Kelamin'] = df_try['Jenis Kelamin'].str.replace('Laki-laki','Laki-Laki')
    df_try = df_try.replace({'Laki-laki':'Laki-Laki','perempuan':'Perempuan'})
    df_try['Anak Ke'] = df_try['Anak Ke -'].str[0]
    df_try['Agama'] = df_try['Agama'].replace(fix_agama, regex=True)
    df_try['Jenis Kelamin'] = df_try['Jenis Kelamin'].str.replace(' ','')
    
    for idx_df,baris in df_try.iterrows():
        print(kelas_sd,idx_df)
        browser.get(link_url)
        wait = WebDriverWait(browser, 30)

        # pilih jenis pendaftaran
        select = Select(browser.find_element(By.XPATH,'//*[@id="card0"]/div/div/div[1]/select'))
        select.select_by_value('1')
        # Pendaftaran Siswa
        browser.find_element(By.XPATH,'//*[@id="accordion"]/div/button').click()
        time.sleep(0.2)
        date= wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@name="date_school"]')))
        browser.execute_script(f"arguments[0].removeAttribute('readonly')", date)
        browser.find_element(By.XPATH,'//*[@name="date_school"]').clear()
        browser.find_element(By.XPATH,'//*[@name="date_school"]').send_keys(baris['Pada Tanggal'])
        browser.find_element(By.XPATH,'//*[@name="school_before"]').clear()
        browser.find_element(By.XPATH,'//*[@name="school_before"]').send_keys(baris['Nama Sekolah Asal'])
        browser.find_element(By.XPATH,'//*[@name="school_address"]').clear()
        browser.find_element(By.XPATH,'//*[@name="school_address"]').send_keys(baris['Alamat Sekolah Asal'])


        # Data Pribadi Siswa
        browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[2]/button').click()
        time.sleep(0.2)
        browser.find_element(By.XPATH,'//*[@name="name"]').clear()
        browser.find_element(By.XPATH,'//*[@name="name"]').send_keys(baris['Nama'])
        
        try:
            NIS = int(baris['No Induk'])
        except:
            NIS = str(baris['No Induk'])
        browser.find_element(By.XPATH,'//*[@name="nis"]').clear()
        browser.find_element(By.XPATH,'//*[@name="nis"]').send_keys(NIS)
        
        try:
            NISN = int(baris['NISN'])
        except:
            NISN = str(baris['NISN'])
        browser.find_element(By.XPATH,'//*[@name="nisn"]').clear()
        browser.find_element(By.XPATH,'//*[@name="nisn"]').send_keys(NISN)
        browser.find_element(By.XPATH,'//*[@name="place_born"]').clear()
        browser.find_element(By.XPATH,'//*[@name="place_born"]').send_keys(baris['Tempat Lahir'])
        date= wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@name="birth_born"]')))
        browser.execute_script(f"arguments[0].removeAttribute('readonly')", date)
        browser.find_element(By.XPATH,'//*[@name="birth_born"]').clear()
        browser.find_element(By.XPATH,'//*[@name="birth_born"]').send_keys(baris['Tanggal Lahir'])
        select = Select(browser.find_element(By.XPATH,'//*[@name="gender"]'))
        select.select_by_visible_text(baris['Jenis Kelamin'].replace('laki','Laki'))
        browser.find_element(By.XPATH,'//*[@name="address"]').clear()
        browser.find_element(By.XPATH,'//*[@name="address"]').send_keys(baris['Nama Ibu'])
        select = Select(browser.find_element(By.XPATH,'//*[@name="child_order"]'))
        select.select_by_visible_text(baris['Anak Ke'])

        # Data Ibu Kandung
        browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[3]/button').click()
        time.sleep(0.2)
        browser.find_element(By.XPATH,'//*[@name="parent[1][name]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[1][name]"]').send_keys(baris['Nama Ibu'])
        browser.find_element(By.XPATH,'//*[@name="parent[1][phone_number]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[1][phone_number]"]').send_keys(str(baris['Telp Ibu']))
        browser.find_element(By.XPATH,'//*[@name="parent[1][work]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[1][work]"]').send_keys(baris['Pekerjaan Ibu'])

        # Data Ayah Kandung
        browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[4]/button').click()
        time.sleep(0.2)
        browser.find_element(By.XPATH,'//*[@name="parent[2][name]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[2][name]"]').send_keys(baris['Nama Ayah'])
        browser.find_element(By.XPATH,'//*[@name="parent[2][phone_number]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[2][phone_number]"]').send_keys(str(baris['Telp Ayah']))
        browser.find_element(By.XPATH,'//*[@name="parent[2][work]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[2][work]"]').send_keys(baris['Pekerjaan Ayah'])

        # Data Wali
        browser.find_element(By.XPATH,'//*[@id="accordion"]/div/div[5]/button').click()
        time.sleep(0.2)
        browser.find_element(By.XPATH,'//*[@name="parent[3][name]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[3][name]"]').send_keys(baris['Nama Wali'])
        browser.find_element(By.XPATH,'//*[@name="parent[3][phone_number]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[3][phone_number]"]').send_keys(str(baris['Telp Wali']))
        browser.find_element(By.XPATH,'//*[@name="parent[3][work]"]').clear()
        browser.find_element(By.XPATH,'//*[@name="parent[3][work]"]').send_keys(baris['Pekerjaan Wali'])

        browser.find_element(By.XPATH,'//*[@id="content"]/form/div/div[2]/button').click()

G10
G10 0
G10 1
G10 2
G10 3
G10 4
G10 5
G10 6
G10 7
G10 8
G10 9
G10 10
G10 11
G11
G11 0
G11 1
G11 2
G11 3
G11 4
G11 5
G11 6
G11 7
G11 8
G11 9
G11 10
G11 11
G11 12
G11 13
